## 학습 목표 

- 머신러닝을 위해서, numpy와 pandas의 사용법을 다시 연습합니다.

- numpy를 사용하여 여러가지 벡터 연산을 연습하고, 실제 데이터 유사도를 계산해봅시다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1. 라이브러리 연습 예제 

[ 필수 RECAP ]

> Numpy : broadcasting, universal function

> Pandas : fancy indexing, categorical variable encoding

In [2]:
import pandas as pd
import numpy as np

In [3]:
# data를 불러와 봅니다.
base_path = '/content/drive/MyDrive/Colab Notebooks/data/prudential-life-insurance-assessment/'
data = pd.read_csv(base_path+'train.csv')

In [4]:
data

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,...,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
0,2,1,D3,10,0.076923,2,1,1,0.641791,0.581818,...,0,0,0,0,0,0,0,0,0,8
1,5,1,A1,26,0.076923,2,3,1,0.059701,0.600000,...,0,0,0,0,0,0,0,0,0,4
2,6,1,E1,26,0.076923,2,3,1,0.029851,0.745455,...,0,0,0,0,0,0,0,0,0,8
3,7,1,D4,10,0.487179,2,3,1,0.164179,0.672727,...,0,0,0,0,0,0,0,0,0,8
4,8,1,D2,26,0.230769,2,3,1,0.417910,0.654545,...,0,0,0,0,0,0,0,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59376,79142,1,D1,10,0.230769,2,3,1,0.074627,0.709091,...,0,0,0,0,0,0,0,0,0,4
59377,79143,1,D3,26,0.230769,2,3,1,0.432836,0.800000,...,0,0,0,0,0,0,0,0,0,7
59378,79144,1,E1,26,0.076923,2,3,1,0.104478,0.745455,...,0,0,0,0,0,0,0,0,0,8
59379,79145,1,D2,10,0.230769,2,3,1,0.507463,0.690909,...,1,0,0,0,0,0,0,0,0,8


### 2. 유사도 측정 예제

#### EDA & Preprocessing

- 퀵하게 데이터를 파악하고, 유사도 측정 예시에 알맞게 가공해봅시다.

In [16]:
## EDA

data = data.iloc[:, :15]

# Ing_Age가 0.5를 넘는 모든 사람
data.loc[data.Ins_Age>0.5, ['Ht','Wt','BMI']]

# col조건, Medical_Keyword라고 시작하는 모든 column을 뽑는다
data.columns[data.columns.str.startswith("Medical_Keyword")]

data

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3
0,2,1,D3,10,0.076923,2,1,1,0.641791,0.581818,0.148536,0.323008,0.028,12,1
1,5,1,A1,26,0.076923,2,3,1,0.059701,0.600000,0.131799,0.272288,0.000,1,3
2,6,1,E1,26,0.076923,2,3,1,0.029851,0.745455,0.288703,0.428780,0.030,9,1
3,7,1,D4,10,0.487179,2,3,1,0.164179,0.672727,0.205021,0.352438,0.042,9,1
4,8,1,D2,26,0.230769,2,3,1,0.417910,0.654545,0.234310,0.424046,0.027,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59376,79142,1,D1,10,0.230769,2,3,1,0.074627,0.709091,0.320084,0.519103,0.020,1,3
59377,79143,1,D3,26,0.230769,2,3,1,0.432836,0.800000,0.403766,0.551119,0.100,9,1
59378,79144,1,E1,26,0.076923,2,3,1,0.104478,0.745455,0.246862,0.360969,0.035,9,1
59379,79145,1,D2,10,0.230769,2,3,1,0.507463,0.690909,0.276151,0.462452,0.038,9,1


In [20]:
# id 컬럼을 drop하고 product_info_2 커럼을 숫자로 변경

# 1. Id drop

# data = data.drop(columns='Id')

# 2. Product_Info_2 를 숫자로 변경 -> Ordinal Encoding, One-Hot Encoding
'''
"매우 싫음"  "싫음"  "보통"  "좋음"  "아주 좋음"  --> Ordinal Encoding
1           2      3      4         5

"남자"   "여자"   "trans"   "중성"   "응답하지 않음" --> One-hot Encoding
1         2       3        4         5
'''
# 2-1 Orinal Encoding (pandas)
data.Product_Info_2 = pd.factorize(data.Product_Info_2)[0]
data

# 2-2 One-Hot Encoding (pandas)
# pd.get_dummies(data=data, columns=["Product_Info_2"]) -> D3만 1이고 나머지는 모두 0



,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3
0,1,0,10,0.076923,2,1,1,0.641791,0.581818,0.148536,0.323008,0.028,12,1
1,1,1,26,0.076923,2,3,1,0.059701,0.600000,0.131799,0.272288,0.000,1,3
2,1,2,26,0.076923,2,3,1,0.029851,0.745455,0.288703,0.428780,0.030,9,1
3,1,3,10,0.487179,2,3,1,0.164179,0.672727,0.205021,0.352438,0.042,9,1
4,1,4,26,0.230769,2,3,1,0.417910,0.654545,0.234310,0.424046,0.027,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59376,1,7,10,0.230769,2,3,1,0.074627,0.709091,0.320084,0.519103,0.020,1,3
59377,1,0,26,0.230769,2,3,1,0.432836,0.800000,0.403766,0.551119,0.100,9,1
59378,1,2,26,0.076923,2,3,1,0.104478,0.745455,0.246862,0.360969,0.035,9,1
59379,1,4,10,0.230769,2,3,1,0.507463,0.690909,0.276151,0.462452,0.038,9,1


#### 유사도(Proximity)

1. Euclidean distance

2. Cosine Similarity

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
v1 = np.array([1,2,3])
v2 = np.array([0,0,0])

In [ ]:
#numpy broadcasting

v1+v2
v1-v2
v1*v2

1/v1

array([1.        , 0.5       , 0.33333333])

In [ ]:
# v1과 v2에 있는 모든 원소들끼리 뺀 다음에, 제곱하고 모두 더해서 루트 씌우기 = Euclidean distancce(L2)

distance = np.sqrt(np.sum(np.square(v1-v2)))

distance2 = np.linalg.norm(v1-v2) # 벡터의 크기 (L2 norm 구하기)

distance == distance2

True

In [22]:
def euclidean_distance(v1, v2):
    return np.sqrt(np.sum(np.square(v1-v2))) 

In [28]:
def cosine_similarity(v1, v2):
    # 1. 분자: a와 b의 내적
    dot = v1 @ v2

    # 2. 분모: |a|, |b|
    norm = np.linalg.norm(v1) * np.linalg.norm(v2)+ 1e-6
    return dot/norm

In [29]:
cosine_similarity(v1, v2)

0.0

#### 6번 고객과 가장 가까운 고객 찾기

In [52]:
len(data)

59381

In [59]:
target = data.loc[2] ## data.iloc[2]

# target을 기준으로 모든 데이터에 대해서 유사도를 계산한 뒤, 가장 유사한 index 출력
# euclidean -> cosine

# result = 0
# for i in range(len(data)):
#     ed = euclidean_distance(target, data.loc[i])
#     result = min(result, ed)
# print(result)
distances=[]

for idx, row in data.iterrows():
    distances.append(euclidean_distance(target, row))

In [62]:
np.argmin(distances)  #argmin: index반환

2

In [63]:
distances[2] = 9999

0.0

In [65]:
np.argmin(distances)

18153